## 03 Feature Extraction

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

#### Check GPU availability

In [2]:
from chosen_gpu import get_freer_gpu
device = torch.device(get_freer_gpu()) 
print("Configured device: ", device)

ModuleNotFoundError: No module named 'chosen_gpu'

### 1. Loading Data

In [3]:
par = "par1"
file = "par1_com2_data_2021-03-27-13.52.44"
task = "_visual_1"

path = 'data/np/{par}/{file}{task}.npy'.format(par=par,file=file, task=task)

X = np.load('data/np/{par}/{file}{task}_X.npy'.format(par=par,file=file, task=task))
y = np.load('data/np/{par}/{file}{task}_y.npy'.format(par=par,file=file, task=task))

FileNotFoundError: [Errno 2] No such file or directory: 'data/np/par1/par1_com2_data_2021-03-27-13.52.44_visual_1_X.npy'

#### 1.1 Check shape

In [ ]:
# [# stim, # electrod, # datapoint]
print(X.shape)
print(y.shape)

#### 1.2 Plot

In [ ]:
### Plot to see wheter eegs have drift or not
data = X.reshape(-1,16)
print(data.shape)
fig, ax = plt.subplots(16,1,figsize=(200,100),sharex=True)

start_point = 3000
plot_lenght = 3200

for i in range(data.shape[1]):
    ax[i].plot(range(plot_lenght),data[start_point:start_point+plot_lenght,i])

### 3.2 Chunking 
- For X : Chunking eeg to lengh of 10 data point in each stimuli's eeg
- For y(lebels) : Filled the lebels in y because we chunk X ( 1 stimuli into 6 chunk). We have 500 labels before but now we need 500 x 6 = 3000 labels

In [ ]:
# chunk_size = 10 
# print(f'Shape of X : {X.shape}')
# print(f'Shape of y : {y.shape}')

In [ ]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

# def chunk_data(data, size):
#     data_keep = data.shape[2] - (data.shape[2]%size)
#     #print(f'{data.shape}')
#     data      = data[:,:,:data_keep]
#     #print(f'{data.shape}')
#     #print(data[0,0,:20])
#     data      = data.reshape(-1,data.shape[1],data.shape[2]//size,size)
#     #print(f'{data.shape}')
#     #print(data[0,0,:2,:])
#     data      = np.transpose(data, (0, 2, 1, 3)  )
#     #print(f'{data.shape}')
#     #print(data[0,:2,0,:])
    
#     return data

# def filled_y(y   , chunk_num):
#     yy = np.array(  [ [ i ] *chunk_num for i in  y     ] ).ravel()

#     return yy



# print(f'Oringinal X shape {X.shape}')
# X_chunk = chunk_data(X, chunk_size)
# X       = X_chunk
# print(f'Reshape X to : {X.shape}')
# chunk_per_stim = X.shape[1]
# X       = X.reshape(-1,16,10)
# print(f'Reshpae X to : {X.shape}')



# print(f'Shape of y : {y.shape}')
# y   = filled_y(y , chunk_per_stim)
# print(f'Shape of new y : {y.shape}')
# print(np.unique(y))

===========================================
### 4. Define Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import fnmatch

In [ ]:
class EEGEncoder(nn.Module):
    '''
    Expected Input Shape: (batch, channels, height , width)
                          (64   , 16      , 1      , 63   )
    '''
    def __init__(self):
        super().__init__()
        
        self.activation = nn.Tanh()
        
        self.conv1 = nn.Sequential(    nn.Conv1d(16, 32, kernel_size=(1,3),   padding=(0,0), stride=(1,1))  ,  self.activation )
        self.conv2 = nn.Sequential(    nn.Conv1d(32, 64, kernel_size=(1,3) ,  padding=(0,0), stride=(1,1))  ,  self.activation )
        self.fc1   = nn.Sequential(    nn.Linear(7808,256),  self.activation ,nn.Dropout(0.1)   ,nn.BatchNorm1d(256)   )
        self.fc2   = nn.Sequential(    nn.Linear(256,128),  self.activation ,nn.Dropout(0.1)   ,nn.BatchNorm1d(128) )
        self.fc3   = nn.Sequential(    nn.Linear(128,64),  self.activation  ,nn.Dropout(0.1)   ,nn.BatchNorm1d(64) )
        self.fc4   = nn.Sequential(    nn.Linear(64,32),  self.activation   ,nn.Dropout(0.1)   ,nn.BatchNorm1d(32) )
        self.fc5   = nn.Sequential(    nn.Linear(32,10)   )

        
        
    
        self.is_debug= False
        
    def encode(self, X):
        
        
        if self.is_debug  : print('--------Convolute--------'); print(X.shape) 
            
        X = self.conv1(X)
        if self.is_debug  : print(X.shape) 
            
        X = self.conv2(X)
        if self.is_debug  : print(X.shape) 
            
        X = X.flatten(start_dim = 1)

        # print(X.shape) 
 
        X = self.fc1(X)
        if self.is_debug : print('--------Flatten--------') ; print(X.shape) 

        X = self.fc2(X)
        if self.is_debug  : print(X.shape) 

        X = self.fc3(X)
        if self.is_debug  : print(X.shape) 
        
        X = self.fc4(X)
        if self.is_debug  : print(X.shape) 

        X = self.fc5(X)
        if self.is_debug  : print(X.shape) 

            
        return X
        
    def forward(self,X):
        X = self.encode(X)
        return X
    
    def get_latent( self, X):
        if self.is_debug  : print('--------Convolute--------'); print(X.shape) 
            
        X = self.conv1(X)
        if self.is_debug  : print(X.shape) 
            
        X = self.conv2(X)
        if self.is_debug  : print(X.shape) 
            
        X = X.flatten(start_dim = 1)
        if self.is_debug  : print('--------Flatten--------') ; print(X.shape) 
 
        X = self.fc1(X)
        if self.is_debug : print('--------Flatten--------') ; print(X.shape) 

        X = self.fc2(X)
        if self.is_debug  : print(X.shape) 

        X = self.fc3(X)
        if self.is_debug  : print(X.shape) 
        
        X = self.fc4(X)
        if self.is_debug  : print(X.shape) 
        
        return X
    
    def classifier(self, latent):
        return self.fc5(latent)
    
# torch_X = torch.from_numpy(X)
# torch_X1 = torch_X.reshape(torch_X.shape[0],torch_X.shape[1],1,torch_X.shape[2]).float()
# model_AE = AE()
# model_AE.is_debug = True
# model_AE.forward(torch_X1)


===========================================
### 5. Define Training function

Define the training process

We set `model.train()` so dropout is applied.

In [ ]:
def train(model, iterator, optimizer, criterion):
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    predicted_list = []
    model.train()
    
    for batch, labels in iterator:
        
        #Move tensors to the configured device
        batch  = batch.to(device)
        labels = labels.to(device)
       
        
        #Forward pass
        outputs = model(batch.float())
        outputs = outputs.to(device)
        
        loss = criterion(outputs, labels).to(device)

        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        #check accuracy
        predictions = model(batch.float())
        _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
        total += labels.size(0)  #keep track of total
        correct += (predicted == labels).sum().item()  #.item() give the raw number
        acc = 100 * (correct / total)
                
        epoch_loss += loss.item()
        epoch_acc = acc
        predicted_list.append(predicted)
        
    return epoch_loss / len(iterator), epoch_acc, predicted_list


def evaluate(model, iterator, criterion):
    
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    predicted_list = []
    labels_list    = []
        
    model.eval()
    
    with torch.no_grad():
    
        for batch, labels in iterator:
            
            #Move tensors to the configured device
            batch = batch.to(device)
            labels = labels.to(device)
            
            #print(labels)
            

            predictions = model(batch.float())
            loss = criterion(predictions, labels)
            
            
            _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
            #print(predicted)
            
            total += labels.size(0)  #keep track of total
            correct += (predicted == labels).sum().item()  #.item() give the raw number
            acc = 100 * (correct / total)
            
            epoch_loss += loss.item()
            epoch_acc += acc
            
            labels_list.append(labels)
            predicted_list.append(predicted)
           

    return epoch_loss / len(iterator), epoch_acc / len(iterator) ,predicted_list, labels_list


# define a time function useful for calculating time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



def do_plot(train_losses, valid_losses):
    plt.figure(figsize=(25,5))
#     clear_output(wait=True)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Valid Loss')
    plt.title('Train and Val loss')
    plt.legend()
    plt.show()

===========================================
### 6. Actual Training for Feature Extraction 


#### 6.1 Define model parameters
- Count model parameters
- optimizer
- loss function
- GPU

In [ ]:
#Create the model from class
model_EEGEncoder = EEGEncoder()
model_EEGEncoder = model_EEGEncoder.float() #define precision as float to reduce running time
models   = [model_EEGEncoder]


#Count the parameters for writing papers
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

for model in models:
    print(f'The model {type(model).__name__} has {count_parameters(model):,} trainable parameters')# Train the model


#### 6.2 Prepare X and y in correct shape

- For X, pytorch (if set batch_first) LSTM requires to be (batch, seq_len, features).  Thus, for us, it should be (100, 75, 16).
- For y, nothing is special
- So let's convert our numpy to pytorch, and then reshape using view

In [ ]:
torch_X = torch.from_numpy(X)
torch_y = torch.from_numpy(y)

torch_X = torch_X


print(torch_X.shape)
print(torch_y.shape)

In [ ]:
print("Original X: ", torch_X.size())

CNN requires the input shape as (batch, channel, height, width)

In [ ]:
torch_X_reshaped = torch_X.reshape(torch_X.shape[0],torch_X.shape[1],1,torch_X.shape[2])
print("Converted X to ", torch_X_reshaped.size())

#### 6.3 Split test train set, and load them into a DataLoader

In [ ]:
from torch.utils.data import TensorDataset

# Define dataset
ds = TensorDataset(torch_X_reshaped, torch_y)

#Train test split
train_size = int(torch_X_reshaped.size()[0] * 0.7)
valid_size = int(torch_X_reshaped.size()[0] * 0.2)
test_size  = torch_X_reshaped.size()[0] - train_size - valid_size

train_set, valid_set, test_set = torch.utils.data.random_split(ds, [train_size, valid_size, test_size])



BATCH_SIZE = 128 #keeping it binary so it fits GPU
#Train set loader
train_iterator = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)
#Validation set loader
valid_iterator = torch.utils.data.DataLoader(dataset=valid_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

#Test set loader
test_iterator = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=test_size, 
                                          shuffle=True)


#### 6.4 Train the model

In [ ]:
import torch.optim as optim

from chosen_gpu import get_freer_gpu
device = torch.device(get_freer_gpu()) 
print("Configured device: ", device)

best_valid_loss = float('inf')
train_losses    = []
valid_losses    = []

learning_rate = 0.001
N_EPOCHS      = 1000          ## best is 10k
criterion     = nn.CrossEntropyLoss()
optimizer     = torch.optim.Adam(model.parameters(), lr=learning_rate)


for model in models:
    model = model.to(device)
criterion = criterion.to(device)


In [ ]:
model.is_debug=False
iteration = 0

for i, model in enumerate(models):
    print(f"Training {type(model).__name__}")
    
    start_time = time.time()

    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc, train_predicted    = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, valid_predicted, _ = evaluate(model, valid_iterator, criterion)

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        iteration     += 1
        
        if (epoch+1) % 50 == 0:
            clear_output(wait=True)
            print(f'Epoch: {epoch+1:02}/{N_EPOCHS}  |',end='')
            print(f'\tTrain Loss: {train_loss:.5f}  | Train Acc: {train_acc:.2f}%  |', end='')
            print(f'\t Val. Loss: {valid_loss:.5f}  | Val. Acc: {valid_acc:.2f}%')
            do_plot(train_losses, valid_losses)
            
                    #--------- Display loss---------
#         if batch_i % print_every == 0:
#             clear_output(wait=True)
#             print('Epoch : {:1d}/{:1d} | Total iteration : {:1d} | d_loss: {:6.6f} | g_loss: {:6.6f}'.format(epoch+1, NUM_EPOCH, iteration, d_loss.item(), g_loss.item()))
            

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            print("Model:{} saved.".format(type(model).__name__))

            if epoch%200 == 0 :
                torch.save(model.state_dict(), "../weights/HCI/feature_extract/{par}{model}{epoch}.pt.tar".format(par=par,model=type(model).__name__,epoch=epoch))
            
            best_model_index = i


### 7. Evaluation (Test model)
using test set

In [ ]:
def squeeze_to_list(_tmp):
    from functools import reduce
    import operator

    xx     = [ i.cpu().detach().numpy().ravel().tolist() for i in _tmp]
    xx     = reduce(operator.concat, xx)
    return xx



models[best_model_index].load_state_dict(torch.load('../weights/HCI/feature_extract/par1EEGEncoder0.pt.tar'))



# test_loss = evaluate(models[best_model_index], test_iterator, criterion)
# print(f'Test Loss: {test_loss}') # | Test Acc: {test_acc:.2f}%')

test_loss, test_acc , predicted, actual_labels = evaluate(models[best_model_index], test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}%')
print("---------------")
print(" (Actual y , Predicted y)")

y_test     = squeeze_to_list(actual_labels)
y_hat_test = squeeze_to_list(predicted)

out = zip(y_test, y_hat_test)

print(list(out))




### 8. Save features extracted

In [ ]:

# encoder_best_model = models[best_model_index].load_state_dict(torch.load(f'save/{type(model).__name__}{i}.pth.tar'))
# torch.save(encoder_best_model, 'save/7.1_encoder_best_model.pth.tar')


### save Encoder network
# torch.save(model_EEGEncoder, 'save/model_EEGEncoder_network_5s.pt.tar')
# torch.save(test_iterator,'save/eeg_X_test_5s.pt.tar')


# save extracted features
eeg_encode = model_EEGEncoder.get_latent(torch_X_reshaped.to(device).float())
eeg_extracted_features = eeg_encode.detach().cpu().numpy()
np.save('data/extracted_features/{par}/{file}{task}_X'.format(par=par,file=file, task=task), eeg_extracted_features )
np.save('data/extracted_features/{par}/{file}{task}_y'.format(par=par,file=file, task=task),y )